In [1]:
from rnnsim.RNN import RNN
import random
import pandas as pd
import numpy as np
import networkx as nx
import sparse

In [2]:
class Traveller:
    def __init__(self, current_node=None, trip_size=10, time = 0):
        self.current_node = current_node
        self.current_step = 0
        self.trip_size = trip_size
        self.first_node = current_node
        self.last_node = None
        self.time = time
        self.timeend = self.time + trip_size
        
    
    def next_step(self, next_nodes):
        if self.is_finished():
            return False
        
        if len(next_nodes) == 0 or self.current_step == self.trip_size:
            self.last_node = self.current_node
            return False
        
        self.current_node = random.choice(next_nodes)
        self.current_step += 1
        return self.current_node
    
    def is_finished(self) -> bool:
        return self.last_node is not None

In [3]:
class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = defaultdict(edges)
        self.distances = {}


def dijsktra(graph, initial):
    visited = {initial: 0}
    path = {}

    nodes = set(graph.nodes)

    while nodes: 
        min_node = None
        for node in nodes:
            if node in visited:
                if min_node is None:
                    min_node = node
                elif visited[node] < visited[min_node]:
                    min_node = node

        if min_node is None:
            break

        nodes.remove(min_node)
        current_weight = visited[min_node]

        for edge in graph.edges[min_node]:
            weight = current_weight + graph.distance[(min_node, edge)]
            if edge not in visited or weight < visited[edge]:
                visited[edge] = weight
                path[edge] = min_node
    return visited, path

In [4]:
def generate_random_graph(nodes_amount=10):
    nodes = set(range(nodes_amount))
    edges = []
    for node in nodes:
        neighbours = random.choices(list(nodes - {node}), k=random.randint(0, len(nodes)))
        for neighbour in neighbours:
            edges.append((node, neighbour, {'weight': 0}))
    
    return nodes, edges

  #### Args:
    graphs_number (int): number of graphs to be generated 
    iteration_number (int): number of iterations in time series
    graph_size (tuple): parameters of even distribution for a number of a graph nodes 
    initial_travellers (int): number of travellers that exist at the first iteration
    new_travellers_on_each_iteration: (tuple): parameters of even distribution for a number of travellers being  added on an each iteration
    trip_size (tuple): parameters of even distribution for steps number in a trip (a trip size is randomly picked for an each traveller)
 #### Returns:
    2 pandas dataframes and list

    ##### The first one contains flows data as rows of edges. It has the following columns:
      graph: id of a graph
      time: time step
      source: a source node id
      target: a target node id
      volume: number of travellers passed by this edge at this time step

    ##### The second one contains trips list. It has the following columns:
      graph: id of a graph
      start_point: node id of a starting point of a trip
      end_point: node id of an end point of a trip
      
    ##### The list contains trip matrix. That means that every element of the list contains the shortest path between two edges.

In [5]:
def generate_graphs(graphs_number=1, 
                    iterations_number=50, 
                    graph_size=(10,30), 
                    initial_travellers=100,
                    new_travellers_on_each_iteration=(0,100),
                    trip_size=(5, 20)):
    graphs = []
    for graph in range(graphs_number): #     G = nx.scale_free_graph(random.randint(*graph_size))
        nodes, edges = generate_random_graph(random.randint(*graph_size))
        G_template = nx.DiGraph()
        G_template.add_nodes_from(nodes)
        G_template.add_edges_from(edges)
  
        temporal_graph = []
        travellers = []
        for t in range(initial_travellers):
            travellers.append(Traveller(random.choice(list(G_template.nodes)), random.randint(*trip_size), time = 0))
        for iteration in range(iterations_number):
        # create a new shapshot
            G = G_template.copy()
        
            # add new travellers
            for t in range(random.randint(*new_travellers_on_each_iteration)):
                travellers.append(Traveller(random.choice(list(G.nodes)), random.randint(*trip_size), time = iterations_number))
            for traveller in travellers:
                if iteration == iterations_number - 1:
                    traveller.last_node = traveller.current_node
                    continue
                node_from = traveller.current_node
                next_node = traveller.next_step([nbr for nbr, _ in G[traveller.current_node].items()])
                if next_node is not False:
                    G.edges[node_from, next_node]['weight'] += 1
        
            # save a snapshot
            temporal_graph.append(G)
        graphs.append({'graph': temporal_graph, 'travellers': travellers})
          
    flows = []
    for graph_id in range(graphs_number):
        temporal_graph = graphs[graph_id]['graph']
        for i in range(iterations_number):
            for source, target in temporal_graph[i].edges:
                flows.append([graph_id, i, source, target, temporal_graph[i].edges[source, target]['weight']])
    flows = pd.DataFrame(data=flows, columns=['graph', 'time', 'source', 'target', 'volume'])
      

    trips = []
    for graph_d in range(graphs_number):
        travellers = graphs[graph_d]['travellers']
        for traveller in travellers:
            trips.append([graph_d, traveller.first_node, traveller.last_node, traveller.time, traveller.timeend])
    trips = pd.DataFrame(data=trips, columns=['graph', 'start_point', 'end_point','time', 'etime'])
    
    trip_matrix = []
    for graph_d in range(graphs_number):
        temporal_graph = graphs[graph_id]['graph']
        i = 0
        for source, target in temporal_graph[i].edges:
            trip_matrix.append(nx.dijkstra_path(temporal_graph[i],source,target))
    
    return flows, trips, trip_matrix

In [6]:
flows, trips, trip_matrix = generate_graphs(graphs_number=1, 
                    iterations_number=2,  
                    trip_size=(5,20))
flows

,graph,time,source,target,volume
0,0,0,0,12,0
1,0,0,0,5,1
2,0,0,0,8,2
3,0,0,0,14,1
4,0,0,0,1,0
...,...,...,...,...,...
283,0,1,20,10,0
284,0,1,20,13,0
285,0,1,20,8,0
286,0,1,20,5,0


In [54]:
del flows['graph']
fl = flows.groupby(['source','target'])['volume'].sum()

In [7]:
trip_matrix

[[0, 12],
 [0, 12, 5],
 [0, 1, 20, 8],
 [0, 1, 14],
 [0, 1],
 [0, 7],
 [0, 7, 16],
 [0, 12, 5, 6],
 [1, 20],
 [1, 0, 7],
 [1, 14],
 [1, 11],
 [1, 0],
 [1, 20, 13],
 [1, 9],
 [1, 2],
 [1, 14, 5],
 [1, 15],
 [1, 14, 17],
 [2, 5, 1, 20],
 [2, 16, 7],
 [2, 16, 3],
 [2, 12],
 [2, 5, 1],
 [2, 12, 9, 18],
 [2, 5, 1, 11],
 [2, 5],
 [2, 12, 9, 14, 17],
 [2, 16],
 [2, 5, 13],
 [2, 12, 9],
 [3, 5, 13, 0, 12],
 [3, 5],
 [4, 2],
 [4, 12, 9],
 [4, 12, 9, 13],
 [4, 0, 7, 16],
 [4, 0, 1, 20, 8],
 [4, 0, 1, 11],
 [4, 0],
 [4, 18],
 [4, 12],
 [4, 3],
 [4, 10],
 [5, 13, 0],
 [5, 15],
 [5, 16],
 [5, 6],
 [5, 13],
 [5, 13, 0, 12],
 [5, 16, 19],
 [5, 2],
 [5, 1],
 [6, 1, 14, 5],
 [6, 1, 20, 18],
 [6, 1, 20, 13, 16],
 [6, 1, 0, 7],
 [6, 1, 14],
 [6, 1],
 [7, 16, 1, 14],
 [7, 19],
 [7, 20, 12],
 [7, 20],
 [7, 20, 18],
 [7, 20, 8],
 [7, 20, 13, 0],
 [7, 16],
 [9, 14, 5],
 [9, 13],
 [9, 14],
 [9, 14, 5, 6],
 [9, 13, 0],
 [9, 1],
 [9, 7],
 [9, 18],
 [9, 3],
 [10, 15],
 [10, 14, 5, 6],
 [10, 14, 5],
 [10, 14, 17]

In [55]:
fl = pd.DataFrame(fl)
fl.head()

volume
source target        
0      2         1039
       3          981
       6         1057
       7         1053
       9         1051

In [131]:
#del trips['graph']
target = []
for path in trip_matrix:
    Y = []
    for time in range(500):
        k = trips[trips.start_point == path[0]][trips.end_point == path[-1]][trips.time <= time][trips.etime >= time].shape[0]
        Y.append(k)
    target.append(Y)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [132]:
target

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [133]:
trips

,start_point,end_point,time,etime
0,6,1,0,16
1,8,10,0,12
2,10,10,0,19
3,21,9,0,5
4,12,10,0,12
...,...,...,...,...
25799,19,19,500,510
25800,21,21,500,518
25801,11,11,500,511
25802,9,9,500,515


In [134]:
print(trip_matrix)

[[0, 6], [0, 21, 7], [0, 20, 3], [0, 16], [0, 13], [0, 21], [0, 21, 12, 15], [0, 6, 11, 2], [0, 20], [0, 9], [0, 21, 17], [1, 0, 9], [1, 0], [1, 12], [1, 16], [1, 18, 3], [1, 14], [1, 12, 10], [1, 0, 6, 5], [1, 19, 2], [1, 13], [1, 19], [1, 17], [1, 18], [2, 13, 20, 16, 6], [2, 13, 20, 3, 4], [2, 13], [3, 10], [3, 20, 16], [3, 12], [3, 20], [3, 4], [3, 12, 17], [3, 8], [3, 1], [3, 15], [3, 0], [3, 12, 19], [3, 11], [3, 12, 9], [4, 11, 1, 17], [4, 11], [4, 5], [4, 5, 12, 15], [4, 7], [4, 11, 9], [4, 19], [4, 16], [4, 7, 18], [4, 5, 6], [4, 11, 1], [4, 20], [4, 10], [5, 21], [5, 14], [5, 13], [5, 1], [5, 14, 9], [5, 21, 17], [5, 12], [5, 21, 19, 2], [5, 3], [5, 6], [5, 3, 4], [5, 7], [5, 8], [5, 12, 15], [6, 20], [6, 20, 9], [6, 14], [6, 5, 12], [6, 5, 21], [6, 8], [6, 11], [6, 5], [7, 10], [7, 18, 19, 11], [7, 18], [7, 18, 19], [7, 18, 14], [8, 4, 20], [8, 18], [8, 18, 19], [8, 14], [8, 4], [8, 4, 16], [9, 3], [9, 14], [9, 18, 2], [9, 3, 0], [9, 10], [9, 12], [9, 3, 0, 21], [9, 7], [9, 

In [188]:
data = []
for path in trip_matrix:
    X = []
    for time in range(500):
        t = 0
        Y = [] #np.zeros((len(path)-1, 3)) # source, target , value
        for i, node in enumerate(path):
            if i == 0:
                #Y[t][0] = node
                pass
            else:
                #Y[t][1] = node
                k = flows[flows.source == path[i-1]][flows.target == node][flows.time == time].index
                if len(path) == 2:
                    Y = flows.loc[k[0],'volume']
                else:
                    Y.append(int(flows.loc[k[0],'volume']))
                if (i != len(path) - 1):
                    t += 1
                    #Y[t][0] = node                         
        X.append(Y)
    data.append(X)      

/usr/lib/python3/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


In [185]:
Y

0

In [187]:
from sklearn.model_selection import train_test_split
data = np.array(data)
target = np.array(target)
print(data[2])
print(target[1])

[0. 0. 0. 0. 1. 0. 2. 2. 0. 2. 3. 1. 2. 1. 2. 1. 0. 1. 0. 0. 4. 1. 1. 0.
 3. 4. 0. 3. 1. 0. 1. 0. 3. 5. 2. 1. 4. 2. 4. 5. 1. 2. 1. 0. 1. 0. 1. 1.
 4. 2. 4. 1. 0. 0. 0. 2. 3. 2. 0. 2. 2. 0. 3. 1. 1. 2. 2. 0. 0. 1. 1. 0.
 2. 1. 4. 2. 3. 1. 0. 0. 2. 0. 1. 2. 2. 1. 3. 0. 2. 3. 4. 2. 0. 2. 0. 1.
 0. 0. 1. 3. 0. 0. 2. 4. 1. 0. 4. 1. 0. 4. 3. 0. 2. 2. 2. 4. 1. 1. 4. 0.
 1. 3. 1. 2. 2. 1. 4. 2. 3. 2. 2. 2. 4. 4. 2. 0. 2. 1. 3. 4. 5. 1. 4. 0.
 0. 4. 2. 1. 2. 1. 3. 2. 1. 0. 5. 0. 3. 1. 5. 3. 2. 3. 2. 1. 2. 3. 3. 1.
 1. 3. 2. 2. 4. 3. 0. 3. 2. 1. 1. 2. 3. 3. 2. 2. 0. 2. 4. 2. 1. 2. 2. 2.
 2. 1. 3. 0. 1. 0. 0. 5. 1. 2. 2. 4. 2. 3. 1. 1. 1. 3. 0. 1. 2. 2. 5. 2.
 2. 3. 4. 3. 3. 4. 2. 3. 0. 0. 2. 1. 2. 5. 2. 1. 3. 1. 2. 3. 2. 0. 4. 3.
 3. 1. 0. 0. 2. 3. 1. 3. 2. 3. 2. 5. 2. 1. 1. 2. 5. 2. 2. 5. 0. 1. 2. 2.
 2. 0. 1. 2. 1. 1. 1. 2. 0. 2. 5. 2. 2. 1. 0. 3. 1. 4. 4. 0. 2. 2. 3. 4.
 2. 1. 6. 3. 5. 2. 2. 2. 3. 1. 0. 2. 1. 1. 1. 1. 3. 4. 1. 0. 5. 0. 2. 5.
 1. 1. 4. 3. 2. 3. 2. 1. 1. 2. 1. 5. 4. 2. 0. 2. 1.

In [191]:
from rnnsim.model import SequentialRNN
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, make_scorer, f1_score, plot_precision_recall_curve, precision_recall_curve
acc = []
for i, path in enumerate(trip_matrix):
    model = SequentialRNN([len(path) - 1, 100, 1])
    model.compile()
    X_train, X_test, y_train, y_test = train_test_split(data[i], target[i], test_size=0.35)
    X_train = np.array(X_train).reshape(1,-1)
    y_train = np.array(y_train).reshape(1,-1)
    X_test = np.array(X_test).reshape(1,-1)
    y_test = np.array(y_test).reshape(1,-1)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    model.fit(train_data=(X_train, y_train), epochs=50, metrics="acc")
    print("RESULTS FOR OD {}-{}".format(path[0],path[1]))
    acc.append(model.score((X_test, y_test)))
    print(model.score((X_test, y_test)))
    

(1, 325) (1, 325) (1, 175) (1, 175)
Iter: 1 Train Accuracy: 100.000 %  Time: 0.030724
Iter: 2 Train Accuracy: 100.000 %  Time: 0.050931
Iter: 3 Train Accuracy: 100.000 %  Time: 0.050564
Iter: 4 Train Accuracy: 100.000 %  Time: 0.049656
Iter: 5 Train Accuracy: 100.000 %  Time: 0.069778
Iter: 6 Train Accuracy: 100.000 %  Time: 0.049276
Iter: 7 Train Accuracy: 100.000 %  Time: 0.049876
Iter: 8 Train Accuracy: 100.000 %  Time: 0.050470
Iter: 9 Train Accuracy: 100.000 %  Time: 0.062458
Iter: 10 Train Accuracy: 100.000 %  Time: 0.055053
Iter: 11 Train Accuracy: 100.000 %  Time: 0.051945
Iter: 12 Train Accuracy: 100.000 %  Time: 0.052131
Iter: 13 Train Accuracy: 100.000 %  Time: 0.075605
Iter: 14 Train Accuracy: 100.000 %  Time: 0.050930
Iter: 15 Train Accuracy: 100.000 %  Time: 0.053482
Iter: 16 Train Accuracy: 100.000 %  Time: 0.058169
Iter: 17 Train Accuracy: 100.000 %  Time: 0.071095
Iter: 18 Train Accuracy: 100.000 %  Time: 0.054176
Iter: 19 Train Accuracy: 100.000 %  Time: 0.074632
Iter

Iter: 8 Train Accuracy: 100.000 %  Time: 0.087439
Iter: 9 Train Accuracy: 100.000 %  Time: 0.077718
Iter: 10 Train Accuracy: 100.000 %  Time: 0.075776
Iter: 11 Train Accuracy: 100.000 %  Time: 0.082601
Iter: 12 Train Accuracy: 100.000 %  Time: 0.084133
Iter: 13 Train Accuracy: 100.000 %  Time: 0.076898
Iter: 14 Train Accuracy: 100.000 %  Time: 0.094372
Iter: 15 Train Accuracy: 100.000 %  Time: 0.076268
Iter: 16 Train Accuracy: 100.000 %  Time: 0.053073
Iter: 17 Train Accuracy: 100.000 %  Time: 0.049578
Iter: 18 Train Accuracy: 100.000 %  Time: 0.058279
Iter: 19 Train Accuracy: 100.000 %  Time: 0.055513
Iter: 20 Train Accuracy: 100.000 %  Time: 0.052864
Iter: 21 Train Accuracy: 100.000 %  Time: 0.053291
Iter: 22 Train Accuracy: 100.000 %  Time: 0.063453
Iter: 23 Train Accuracy: 100.000 %  Time: 0.053555
Iter: 24 Train Accuracy: 100.000 %  Time: 0.053730
Iter: 25 Train Accuracy: 100.000 %  Time: 0.054890
Iter: 26 Train Accuracy: 100.000 %  Time: 0.060313
Iter: 27 Train Accuracy: 100.000 

Iter: 15 Train Accuracy: 100.000 %  Time: 0.123621
Iter: 16 Train Accuracy: 100.000 %  Time: 0.105482
Iter: 17 Train Accuracy: 100.000 %  Time: 0.111249
Iter: 18 Train Accuracy: 100.000 %  Time: 0.112195
Iter: 19 Train Accuracy: 100.000 %  Time: 0.138295
Iter: 20 Train Accuracy: 100.000 %  Time: 0.100349
Iter: 21 Train Accuracy: 100.000 %  Time: 0.129730
Iter: 22 Train Accuracy: 100.000 %  Time: 0.178175
Iter: 23 Train Accuracy: 100.000 %  Time: 0.132955
Iter: 24 Train Accuracy: 100.000 %  Time: 0.104450
Iter: 25 Train Accuracy: 100.000 %  Time: 0.109846
Iter: 26 Train Accuracy: 100.000 %  Time: 0.123709
Iter: 27 Train Accuracy: 100.000 %  Time: 0.110868
Iter: 28 Train Accuracy: 100.000 %  Time: 0.095538
Iter: 29 Train Accuracy: 100.000 %  Time: 0.118836
Iter: 30 Train Accuracy: 100.000 %  Time: 0.076961
Iter: 31 Train Accuracy: 100.000 %  Time: 0.087777
Iter: 32 Train Accuracy: 100.000 %  Time: 0.092640
Iter: 33 Train Accuracy: 100.000 %  Time: 0.087100
Iter: 34 Train Accuracy: 100.00

Iter: 24 Train Accuracy: 100.000 %  Time: 0.050252
Iter: 25 Train Accuracy: 100.000 %  Time: 0.047641
Iter: 26 Train Accuracy: 100.000 %  Time: 0.052140
Iter: 27 Train Accuracy: 100.000 %  Time: 0.060171
Iter: 28 Train Accuracy: 100.000 %  Time: 0.068165
Iter: 29 Train Accuracy: 100.000 %  Time: 0.070461
Iter: 30 Train Accuracy: 100.000 %  Time: 0.123033
Iter: 31 Train Accuracy: 100.000 %  Time: 0.061480
Iter: 32 Train Accuracy: 100.000 %  Time: 0.078717
Iter: 33 Train Accuracy: 100.000 %  Time: 0.086758
Iter: 34 Train Accuracy: 100.000 %  Time: 0.133262
Iter: 35 Train Accuracy: 100.000 %  Time: 0.108495
Iter: 36 Train Accuracy: 100.000 %  Time: 0.096265
Iter: 37 Train Accuracy: 100.000 %  Time: 0.102737
Iter: 38 Train Accuracy: 100.000 %  Time: 0.089128
Iter: 39 Train Accuracy: 100.000 %  Time: 0.063839
Iter: 40 Train Accuracy: 100.000 %  Time: 0.069153
Iter: 41 Train Accuracy: 100.000 %  Time: 0.040971
Iter: 42 Train Accuracy: 100.000 %  Time: 0.075919
Iter: 43 Train Accuracy: 100.00

Iter: 31 Train Accuracy: 100.000 %  Time: 0.073288
Iter: 32 Train Accuracy: 100.000 %  Time: 0.059463
Iter: 33 Train Accuracy: 100.000 %  Time: 0.061502
Iter: 34 Train Accuracy: 100.000 %  Time: 0.069602
Iter: 35 Train Accuracy: 100.000 %  Time: 0.071787
Iter: 36 Train Accuracy: 100.000 %  Time: 0.058311
Iter: 37 Train Accuracy: 100.000 %  Time: 0.058339
Iter: 38 Train Accuracy: 100.000 %  Time: 0.059228
Iter: 39 Train Accuracy: 100.000 %  Time: 0.067893
Iter: 40 Train Accuracy: 100.000 %  Time: 0.057654
Iter: 41 Train Accuracy: 100.000 %  Time: 0.056556
Iter: 42 Train Accuracy: 100.000 %  Time: 0.056107
Iter: 43 Train Accuracy: 100.000 %  Time: 0.063871
Iter: 44 Train Accuracy: 100.000 %  Time: 0.057617
Iter: 45 Train Accuracy: 100.000 %  Time: 0.055747
Iter: 46 Train Accuracy: 100.000 %  Time: 0.056981
Iter: 47 Train Accuracy: 100.000 %  Time: 0.060842
Iter: 48 Train Accuracy: 100.000 %  Time: 0.057303
Iter: 49 Train Accuracy: 100.000 %  Time: 0.055802
Iter: 50 Train Accuracy: 100.00

...

Iter: 7 Train Accuracy: 100.000 %  Time: 0.069303
Iter: 8 Train Accuracy: 100.000 %  Time: 0.069194
Iter: 9 Train Accuracy: 100.000 %  Time: 0.072828
Iter: 10 Train Accuracy: 100.000 %  Time: 0.073779
Iter: 11 Train Accuracy: 100.000 %  Time: 0.068917
Iter: 12 Train Accuracy: 100.000 %  Time: 0.071030
Iter: 13 Train Accuracy: 100.000 %  Time: 0.071323
Iter: 14 Train Accuracy: 100.000 %  Time: 0.073311
Iter: 15 Train Accuracy: 100.000 %  Time: 0.073438
Iter: 16 Train Accuracy: 100.000 %  Time: 0.072244
Iter: 17 Train Accuracy: 100.000 %  Time: 0.073597
Iter: 18 Train Accuracy: 100.000 %  Time: 0.069848
Iter: 19 Train Accuracy: 100.000 %  Time: 0.074561
Iter: 20 Train Accuracy: 100.000 %  Time: 0.070286
Iter: 21 Train Accuracy: 100.000 %  Time: 0.070153
Iter: 22 Train Accuracy: 100.000 %  Time: 0.071860
Iter: 23 Train Accuracy: 100.000 %  Time: 0.071513
Iter: 24 Train Accuracy: 100.000 %  Time: 0.073694
Iter: 25 Train Accuracy: 100.000 %  Time: 0.102440
Iter: 26 Train Accuracy: 100.000 %

Iter: 16 Train Accuracy: 100.000 %  Time: 0.056011
Iter: 17 Train Accuracy: 100.000 %  Time: 0.062079
Iter: 18 Train Accuracy: 100.000 %  Time: 0.057366
Iter: 19 Train Accuracy: 100.000 %  Time: 0.058896
Iter: 20 Train Accuracy: 100.000 %  Time: 0.059578
Iter: 21 Train Accuracy: 100.000 %  Time: 0.057693
Iter: 22 Train Accuracy: 100.000 %  Time: 0.060911
Iter: 23 Train Accuracy: 100.000 %  Time: 0.051946
Iter: 24 Train Accuracy: 100.000 %  Time: 0.056440
Iter: 25 Train Accuracy: 100.000 %  Time: 0.058610
Iter: 26 Train Accuracy: 100.000 %  Time: 0.059333
Iter: 27 Train Accuracy: 100.000 %  Time: 0.066358
Iter: 28 Train Accuracy: 100.000 %  Time: 0.059144
Iter: 29 Train Accuracy: 100.000 %  Time: 0.055159
Iter: 30 Train Accuracy: 100.000 %  Time: 0.060088
Iter: 31 Train Accuracy: 100.000 %  Time: 0.059457
Iter: 32 Train Accuracy: 100.000 %  Time: 0.061827
Iter: 33 Train Accuracy: 100.000 %  Time: 0.053011
Iter: 34 Train Accuracy: 100.000 %  Time: 0.053763
Iter: 35 Train Accuracy: 100.00

Iter: 25 Train Accuracy: 100.000 %  Time: 0.076735
Iter: 26 Train Accuracy: 100.000 %  Time: 0.074545
Iter: 27 Train Accuracy: 100.000 %  Time: 0.075475
Iter: 28 Train Accuracy: 100.000 %  Time: 0.077924
Iter: 29 Train Accuracy: 100.000 %  Time: 0.075668
Iter: 30 Train Accuracy: 100.000 %  Time: 0.076621
Iter: 31 Train Accuracy: 100.000 %  Time: 0.076888
Iter: 32 Train Accuracy: 100.000 %  Time: 0.074519
Iter: 33 Train Accuracy: 100.000 %  Time: 0.076289
Iter: 34 Train Accuracy: 100.000 %  Time: 0.077853
Iter: 35 Train Accuracy: 100.000 %  Time: 0.049141
Iter: 36 Train Accuracy: 100.000 %  Time: 0.050085
Iter: 37 Train Accuracy: 100.000 %  Time: 0.049728
Iter: 38 Train Accuracy: 100.000 %  Time: 0.049204
Iter: 39 Train Accuracy: 100.000 %  Time: 0.049013
Iter: 40 Train Accuracy: 100.000 %  Time: 0.049619
Iter: 41 Train Accuracy: 100.000 %  Time: 0.051110
Iter: 42 Train Accuracy: 100.000 %  Time: 0.050427
Iter: 43 Train Accuracy: 100.000 %  Time: 0.050855
Iter: 44 Train Accuracy: 100.00

Iter: 35 Train Accuracy: 100.000 %  Time: 0.052071
Iter: 36 Train Accuracy: 100.000 %  Time: 0.051866
Iter: 37 Train Accuracy: 100.000 %  Time: 0.052353
Iter: 38 Train Accuracy: 100.000 %  Time: 0.051574
Iter: 39 Train Accuracy: 100.000 %  Time: 0.052791
Iter: 40 Train Accuracy: 100.000 %  Time: 0.053157
Iter: 41 Train Accuracy: 100.000 %  Time: 0.051169
Iter: 42 Train Accuracy: 100.000 %  Time: 0.057980
Iter: 43 Train Accuracy: 100.000 %  Time: 0.075869
Iter: 44 Train Accuracy: 100.000 %  Time: 0.074434
Iter: 45 Train Accuracy: 100.000 %  Time: 0.079189
Iter: 46 Train Accuracy: 100.000 %  Time: 0.075863
Iter: 47 Train Accuracy: 100.000 %  Time: 0.074299
Iter: 48 Train Accuracy: 100.000 %  Time: 0.074891
Iter: 49 Train Accuracy: 100.000 %  Time: 0.072500
Iter: 50 Train Accuracy: 100.000 %  Time: 0.071470
RESULTS FOR OD 21-12
Test Accuracy: 100.000 %
(1, 325) (1, 325) (1, 175) (1, 175)
Iter: 1 Train Accuracy: 100.000 %  Time: 0.040173
Iter: 2 Train Accuracy: 100.000 %  Time: 0.072000
It

Iter: 45 Train Accuracy: 100.000 %  Time: 0.052196
Iter: 46 Train Accuracy: 100.000 %  Time: 0.055013
Iter: 47 Train Accuracy: 100.000 %  Time: 0.061592
Iter: 48 Train Accuracy: 100.000 %  Time: 0.052200
Iter: 49 Train Accuracy: 100.000 %  Time: 0.063810
Iter: 50 Train Accuracy: 100.000 %  Time: 0.053361
RESULTS FOR OD 21-5
Test Accuracy: 100.000 %
(1, 325) (1, 325) (1, 175) (1, 175)
Iter: 1 Train Accuracy: 100.000 %  Time: 0.036275
Iter: 2 Train Accuracy: 100.000 %  Time: 0.062144
Iter: 3 Train Accuracy: 100.000 %  Time: 0.052399
Iter: 4 Train Accuracy: 100.000 %  Time: 0.052772
Iter: 5 Train Accuracy: 100.000 %  Time: 0.052305
Iter: 6 Train Accuracy: 100.000 %  Time: 0.060935
Iter: 7 Train Accuracy: 100.000 %  Time: 0.053232
Iter: 8 Train Accuracy: 100.000 %  Time: 0.052391
Iter: 9 Train Accuracy: 100.000 %  Time: 0.052588
Iter: 10 Train Accuracy: 100.000 %  Time: 0.059089
Iter: 11 Train Accuracy: 100.000 %  Time: 0.047853
Iter: 12 Train Accuracy: 100.000 %  Time: 0.052314
Iter: 13 T

In [177]:
X_train, X_test, y_train, y_test = train_test_split(data[1], target[1], test_size=0.35)
X_train

array([list([3, 1]), list([1, 3]), list([3, 2]), list([2, 0]),
       list([5, 0]), list([4, 1]), list([1, 1]), list([3, 0]),
       list([0, 2]), list([1, 2]), list([2, 0]), list([3, 1]),
       list([2, 2]), list([3, 3]), list([5, 0]), list([0, 1]),
       list([3, 0]), list([2, 3]), list([1, 3]), list([3, 2]),
       list([2, 2]), list([1, 2]), list([2, 0]), list([4, 0]),
       list([4, 1]), list([2, 3]), list([3, 0]), list([2, 0]),
       list([3, 0]), list([4, 1]), list([3, 0]), list([1, 2]),
       list([1, 4]), list([3, 2]), list([3, 2]), list([1, 1]),
       list([3, 2]), list([2, 1]), list([2, 2]), list([2, 0]),
       list([2, 2]), list([0, 1]), list([2, 2]), list([3, 1]),
       list([1, 0]), list([2, 1]), list([6, 0]), list([0, 1]),
       list([2, 0]), list([2, 2]), list([1, 0]), list([1, 2]),
       list([3, 1]), list([2, 1]), list([2, 2]), list([3, 0]),
       list([3, 1]), list([0, 1]), list([1, 2]), list([0, 0]),
       list([1, 0]), list([3, 1]), list([2, 3]), list([

In [192]:
acc_copy = acc


In [197]:
acc = np.array(acc)
len(acc)

184

In [200]:
1 - 26/len(trip_matrix)

0.8586956521739131

In [203]:
acc = []
conn_plus = {
    1: [3, 4], 2: [3, 4],
    3: [5], 4: [5], 5: []}
conn_minus = {
    1: [3, 4], 2: [3, 4],
    3: [5], 4: [5], 5: []}
for i, path in enumerate(trip_matrix):
    model = SequentialRNN([len(path) - 1, 100, 1])
    model.compile()
    X_train, X_test, y_train, y_test = train_test_split(data[i], target[i], test_size=0.35)
    X_train = np.array(X_train).reshape(1,-1)
    y_train = np.array(y_train).reshape(1,-1)
    X_test = np.array(X_test).reshape(1,-1)
    y_test = np.array(y_test).reshape(1,-1)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    
    model = RNN(n_total=5, input_neurons=[len(path)-1], output_neurons=[1], conn_plus=conn_plus, conn_minus=conn_minus)
    model.fit(train_data=(X_train, y_train), epochs=50)
    print("RESULTS FOR OD {}-{}".format(path[0],path[1]))
    acc.append(model.score((X_test, y_test)))
    print(model.score((X_test, y_test)))
    

(1, 325) (1, 325) (1, 175) (1, 175)
Iter: 1 Train Error: 1.00000  Time: 0.000638
Iter: 2 Train Error: 1.00000  Time: 0.000459
Iter: 3 Train Error: 1.00000  Time: 0.007824
Iter: 4 Train Error: 1.00000  Time: 0.000592
Iter: 5 Train Error: 1.00000  Time: 0.000468
Iter: 6 Train Error: 1.00000  Time: 0.000779
Iter: 7 Train Error: 1.00000  Time: 0.000454
Iter: 8 Train Error: 1.00000  Time: 0.000421
Iter: 9 Train Error: 1.00000  Time: 0.000456
Iter: 10 Train Error: 1.00000  Time: 0.000433
Iter: 11 Train Error: 1.00000  Time: 0.000514
Iter: 12 Train Error: 1.00000  Time: 0.000497
Iter: 13 Train Error: 1.00000  Time: 0.000414
Iter: 14 Train Error: 1.00000  Time: 0.000412
Iter: 15 Train Error: 1.00000  Time: 0.000452
Iter: 16 Train Error: 1.00000  Time: 0.000442
Iter: 17 Train Error: 1.00000  Time: 0.000430
Iter: 18 Train Error: 1.00000  Time: 0.000451
Iter: 19 Train Error: 1.00000  Time: 0.000417
Iter: 20 Train Error: 1.00000  Time: 0.000413
Iter: 21 Train Error: 1.00000  Time: 0.000434
Iter: 2

Iter: 8 Train Error: 1.00000  Time: 0.000493
Iter: 9 Train Error: 1.00000  Time: 0.000509
Iter: 10 Train Error: 1.00000  Time: 0.000476
Iter: 11 Train Error: 1.00000  Time: 0.000531
Iter: 12 Train Error: 1.00000  Time: 0.000484
Iter: 13 Train Error: 1.00000  Time: 0.000478
Iter: 14 Train Error: 1.00000  Time: 0.000485
Iter: 15 Train Error: 1.00000  Time: 0.000539
Iter: 16 Train Error: 1.00000  Time: 0.000537
Iter: 17 Train Error: 1.00000  Time: 0.000475
Iter: 18 Train Error: 1.00000  Time: 0.000476
Iter: 19 Train Error: 1.00000  Time: 0.000484
Iter: 20 Train Error: 1.00000  Time: 0.000511
Iter: 21 Train Error: 1.00000  Time: 0.000563
Iter: 22 Train Error: 1.00000  Time: 0.000534
Iter: 23 Train Error: 1.00000  Time: 0.000480
Iter: 24 Train Error: 1.00000  Time: 0.000481
Iter: 25 Train Error: 1.00000  Time: 0.000489
Iter: 26 Train Error: 1.00000  Time: 0.000537
Iter: 27 Train Error: 1.00000  Time: 0.000544
Iter: 28 Train Error: 1.00000  Time: 0.000480
Iter: 29 Train Error: 1.00000  Time:

Iter: 33 Train Error: 1.00000  Time: 0.000513
Iter: 34 Train Error: 1.00000  Time: 0.000495
Iter: 35 Train Error: 1.00000  Time: 0.000497
Iter: 36 Train Error: 1.00000  Time: 0.000517
Iter: 37 Train Error: 1.00000  Time: 0.000504
Iter: 38 Train Error: 1.00000  Time: 0.000506
Iter: 39 Train Error: 1.00000  Time: 0.000524
Iter: 40 Train Error: 1.00000  Time: 0.000525
Iter: 41 Train Error: 1.00000  Time: 0.000485
Iter: 42 Train Error: 1.00000  Time: 0.000537
Iter: 43 Train Error: 1.00000  Time: 0.000554
Iter: 44 Train Error: 1.00000  Time: 0.000503
Iter: 45 Train Error: 1.00000  Time: 0.000495
Iter: 46 Train Error: 1.00000  Time: 0.000509
Iter: 47 Train Error: 1.00000  Time: 0.000535
Iter: 48 Train Error: 1.00000  Time: 0.000519
Iter: 49 Train Error: 1.00000  Time: 0.000527
Iter: 50 Train Error: 1.00000  Time: 0.000486
RESULTS FOR OD 1-16
Error: 1.000 %
(1, 650) (1, 325) (1, 350) (1, 175)
Iter: 1 Train Error: 1.00000  Time: 0.000539
Iter: 2 Train Error: 1.00000  Time: 0.000489
Iter: 3 Tra

/home/magistrkoljan/.local/lib/python3.6/site-packages/rnnsim/RNN.py:176: RuntimeWarning: divide by zero encountered in true_divide
  ) / self.D[i]
/home/magistrkoljan/.local/lib/python3.6/site-packages/rnnsim/RNN.py:176: RuntimeWarning: invalid value encountered in true_divide
  ) / self.D[i]
/home/magistrkoljan/.local/lib/python3.6/site-packages/rnnsim/RNN.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  gammaplus[i] = 1.0 / self.D[i]
/home/magistrkoljan/.local/lib/python3.6/site-packages/rnnsim/RNN.py:200: RuntimeWarning: invalid value encountered in greater_equal
  wplus_result = np.multiply(wplus_result, (wplus_result >= 0.0))
/home/magistrkoljan/.local/lib/python3.6/site-packages/rnnsim/RNN.py:211: RuntimeWarning: divide by zero encountered in double_scalars
  gammaminus[i] = -self.q[i] / self.D[i]



(1, 325) (1, 325) (1, 175) (1, 175)
Iter: 1 Train Error: 1.00000  Time: 0.000574
Iter: 2 Train Error: 1.00000  Time: 0.000606
Iter: 3 Train Error: 1.00000  Time: 0.000500
Iter: 4 Train Error: 1.00000  Time: 0.000495
Iter: 5 Train Error: 1.00000  Time: 0.000489
Iter: 6 Train Error: 1.00000  Time: 0.000542
Iter: 7 Train Error: 1.00000  Time: 0.000555
Iter: 8 Train Error: 1.00000  Time: 0.000521
Iter: 9 Train Error: 1.00000  Time: 0.000494
Iter: 10 Train Error: 1.00000  Time: 0.000486
Iter: 11 Train Error: 1.00000  Time: 0.000481
Iter: 12 Train Error: 1.00000  Time: 0.000662
Iter: 13 Train Error: 1.00000  Time: 0.000592
Iter: 14 Train Error: 1.00000  Time: 0.000489
Iter: 15 Train Error: 1.00000  Time: 0.000481
Iter: 16 Train Error: 1.00000  Time: 0.000482
Iter: 17 Train Error: 1.00000  Time: 0.000618
Iter: 18 Train Error: 1.00000  Time: 0.000616
Iter: 19 Train Error: 1.00000  Time: 0.000491
Iter: 20 Train Error: 1.00000  Time: 0.000486
Iter: 21 Train Error: 1.00000  Time: 0.000484
Iter: 

...

Iter: 9 Train Error: 1.00000  Time: 0.000813
Iter: 10 Train Error: 1.00000  Time: 0.000912
Iter: 11 Train Error: 1.00000  Time: 0.000811
Iter: 12 Train Error: 1.00000  Time: 0.000831
Iter: 13 Train Error: 1.00000  Time: 0.000811
Iter: 14 Train Error: 1.00000  Time: 0.000814
Iter: 15 Train Error: 1.00000  Time: 0.000811
Iter: 16 Train Error: 1.00000  Time: 0.000848
Iter: 17 Train Error: 1.00000  Time: 0.000814
Iter: 18 Train Error: 1.00000  Time: 0.000809
Iter: 19 Train Error: 1.00000  Time: 0.000825
Iter: 20 Train Error: 1.00000  Time: 0.000831
Iter: 21 Train Error: 1.00000  Time: 0.000842
Iter: 22 Train Error: 1.00000  Time: 0.000801
Iter: 23 Train Error: 1.00000  Time: 0.000816
Iter: 24 Train Error: 1.00000  Time: 0.000813
Iter: 25 Train Error: 1.00000  Time: 0.000840
Iter: 26 Train Error: 1.00000  Time: 0.000812
Iter: 27 Train Error: 1.00000  Time: 0.000810
Iter: 28 Train Error: 1.00000  Time: 0.000811
Iter: 29 Train Error: 1.00000  Time: 0.000832
Iter: 30 Train Error: 1.00000  Time

Error: 0.000 %
(1, 325) (1, 325) (1, 175) (1, 175)
Iter: 1 Train Error: 1.00000  Time: 0.000506
Iter: 2 Train Error: 1.00000  Time: 0.000495
Iter: 3 Train Error: 1.00000  Time: 0.000497
Iter: 4 Train Error: 1.00000  Time: 0.000531
Iter: 5 Train Error: 1.00000  Time: 0.000537
Iter: 6 Train Error: 1.00000  Time: 0.000509
Iter: 7 Train Error: 1.00000  Time: 0.000502
Iter: 8 Train Error: 1.00000  Time: 0.000478
Iter: 9 Train Error: 1.00000  Time: 0.000492
Iter: 10 Train Error: 1.00000  Time: 0.000541
Iter: 11 Train Error: 1.00000  Time: 0.000535
Iter: 12 Train Error: 1.00000  Time: 0.000489
Iter: 13 Train Error: 1.00000  Time: 0.000517
Iter: 14 Train Error: 1.00000  Time: 0.000519
Iter: 15 Train Error: 1.00000  Time: 0.000501
Iter: 16 Train Error: 1.00000  Time: 0.000492
Iter: 17 Train Error: 1.00000  Time: 0.000510
Iter: 18 Train Error: 1.00000  Time: 0.000533
Iter: 19 Train Error: 1.00000  Time: 0.000502
Iter: 20 Train Error: 1.00000  Time: 0.000498
Iter: 21 Train Error: 1.00000  Time: 0

Iter: 22 Train Error: 1.00000  Time: 0.000552
Iter: 23 Train Error: 1.00000  Time: 0.000543
Iter: 24 Train Error: 1.00000  Time: 0.000484
Iter: 25 Train Error: 1.00000  Time: 0.000479
Iter: 26 Train Error: 1.00000  Time: 0.000558
Iter: 27 Train Error: 1.00000  Time: 0.000533
Iter: 28 Train Error: 1.00000  Time: 0.000478
Iter: 29 Train Error: 1.00000  Time: 0.000477
Iter: 30 Train Error: 1.00000  Time: 0.000484
Iter: 31 Train Error: 1.00000  Time: 0.000542
Iter: 32 Train Error: 1.00000  Time: 0.000563
Iter: 33 Train Error: 1.00000  Time: 0.000501
Iter: 34 Train Error: 1.00000  Time: 0.000478
Iter: 35 Train Error: 1.00000  Time: 0.000478
Iter: 36 Train Error: 1.00000  Time: 0.000516
Iter: 37 Train Error: 1.00000  Time: 0.000560
Iter: 38 Train Error: 1.00000  Time: 0.000529
Iter: 39 Train Error: 1.00000  Time: 0.000489
Iter: 40 Train Error: 1.00000  Time: 0.000484
Iter: 41 Train Error: 1.00000  Time: 0.000482
Iter: 42 Train Error: 1.00000  Time: 0.000576
Iter: 43 Train Error: 1.00000  Tim

Iter: 2 Train Error: 1.00000  Time: 0.000511
Iter: 3 Train Error: 1.00000  Time: 0.000585
Iter: 4 Train Error: 1.00000  Time: 0.000558
Iter: 5 Train Error: 1.00000  Time: 0.000496
Iter: 6 Train Error: 1.00000  Time: 0.000508
Iter: 7 Train Error: 1.00000  Time: 0.000575
Iter: 8 Train Error: 1.00000  Time: 0.000556
Iter: 9 Train Error: 1.00000  Time: 0.000482
Iter: 10 Train Error: 1.00000  Time: 0.000481
Iter: 11 Train Error: 1.00000  Time: 0.000473
Iter: 12 Train Error: 1.00000  Time: 0.000570
Iter: 13 Train Error: 1.00000  Time: 0.000563
Iter: 14 Train Error: 1.00000  Time: 0.000485
Iter: 15 Train Error: 1.00000  Time: 0.000487
Iter: 16 Train Error: 1.00000  Time: 0.000481
Iter: 17 Train Error: 1.00000  Time: 0.000528
Iter: 18 Train Error: 1.00000  Time: 0.000565
Iter: 19 Train Error: 1.00000  Time: 0.000513
Iter: 20 Train Error: 1.00000  Time: 0.000491
Iter: 21 Train Error: 1.00000  Time: 0.000489
Iter: 22 Train Error: 1.00000  Time: 0.000493
Iter: 23 Train Error: 1.00000  Time: 0.000

Iter: 28 Train Error: 1.00000  Time: 0.000503
Iter: 29 Train Error: 1.00000  Time: 0.000509
Iter: 30 Train Error: 1.00000  Time: 0.000511
Iter: 31 Train Error: 1.00000  Time: 0.000522
Iter: 32 Train Error: 1.00000  Time: 0.000526
Iter: 33 Train Error: 1.00000  Time: 0.000532
Iter: 34 Train Error: 1.00000  Time: 0.000492
Iter: 35 Train Error: 1.00000  Time: 0.000508
Iter: 36 Train Error: 1.00000  Time: 0.000519
Iter: 37 Train Error: 1.00000  Time: 0.000507
Iter: 38 Train Error: 1.00000  Time: 0.000507
Iter: 39 Train Error: 1.00000  Time: 0.000541
Iter: 40 Train Error: 1.00000  Time: 0.000518
Iter: 41 Train Error: 1.00000  Time: 0.000524
Iter: 42 Train Error: 1.00000  Time: 0.000507
Iter: 43 Train Error: 1.00000  Time: 0.000610
Iter: 44 Train Error: 1.00000  Time: 0.000505
Iter: 45 Train Error: 1.00000  Time: 0.000509
Iter: 46 Train Error: 1.00000  Time: 0.000503
Iter: 47 Train Error: 1.00000  Time: 0.000494
Iter: 48 Train Error: 1.00000  Time: 0.000508
Iter: 49 Train Error: 1.00000  Tim

In [204]:
acc

['Error: 0.960 %',
 'Error: 1.000 %',
 'Error: 0.954 %',
 'Error: 1.000 %',
 'Error: 0.000 %',
 'Error: 0.000 %',
 'Error: 0.977 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 0.989 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 0.000 %',
 'Error: 0.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.120 %',
 'Error: 0.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 0.960 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 0.040 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 0.000 %',
 'Error: 0.971 %',
 'Error: 1.000 %',
 'Error: 0.983 %',
 'Error: 0.023 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 0.000 %',
 'Error: 1.000 %',
 'Error: 1.000 %',
 'Error: 0.9

In [209]:
161.345/len(acc)

0.876875